In [2]:
import torch
import random
import pandas as pd
import json
import numpy as np
import torch.backends.cudnn as cudnn

In [3]:
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(seed)
# seed 결과가달라짐 3~4%

device = "cuda:0" if torch.cuda.is_available() else "cpu"
print('device:', device)

device: cuda:0


In [4]:
print('-'*10)
print('Data Loading Start!!')
print('-'*10)

## dataset class
path = "/data/ephemeral/home/data/aug_data_x10.jsonl"

with open(path) as f:
    data = [json.loads(line) for line in f]
data = pd.DataFrame(data)
print('origin_data:', data.shape)


----------
Data Loading Start!!
----------
origin_data: (42720, 5)


In [5]:
data.columns

Index(['docid', 'question', 'content', 'src', 'new_domains'], dtype='object')

In [6]:
data['new_domains'].unique()

array(['human_aging', 'medical_genetics', 'high_school_biology',
       'college_chemistry', 'college_physics', 'conceptual_physics',
       'global_facts', 'None', 'unknown', 'computer_security',
       'high_school_chemistry', 'anatomy', 'nutrition', 'human_sexuality',
       'astronomy', 'high_school_computer_science', 'virology',
       'electrical_engineering', 'college_medicine', 'college_biology',
       'college_computer_science', 'human_aging, nutrition',
       'high_school_physics', 'geology', 'art', 'college_science',
       'safety', 'civil_engineering', 'new_technology_in_industry',
       'engineering', 'data_visualization',
       'human_sexuality, medical_genetics', 'environmental_science',
       'music_performance', 'logistics', 'None, medical_genetics',
       'astronomy, college_physics', 'construction_tools',
       'astronomy, college_biology', 'astronomy, conceptual_physics',
       'anatomy, college_chemistry', 'safety_education',
       'investigation_records'

In [7]:
data2 = data[~data['new_domains'].isin(['None','unknown'])]

In [8]:
data2.shape

(41580, 5)

In [9]:
data2['new_domains'].unique()

array(['human_aging', 'medical_genetics', 'high_school_biology',
       'college_chemistry', 'college_physics', 'conceptual_physics',
       'global_facts', 'computer_security', 'high_school_chemistry',
       'anatomy', 'nutrition', 'human_sexuality', 'astronomy',
       'high_school_computer_science', 'virology',
       'electrical_engineering', 'college_medicine', 'college_biology',
       'college_computer_science', 'human_aging, nutrition',
       'high_school_physics', 'geology', 'art', 'college_science',
       'safety', 'civil_engineering', 'new_technology_in_industry',
       'engineering', 'data_visualization',
       'human_sexuality, medical_genetics', 'environmental_science',
       'music_performance', 'logistics', 'None, medical_genetics',
       'astronomy, college_physics', 'construction_tools',
       'astronomy, college_biology', 'astronomy, conceptual_physics',
       'anatomy, college_chemistry', 'safety_education',
       'investigation_records'], dtype=object)

In [127]:
data2[data2['new_domains']=='astronomy, college_biology']

,docid,question,content,src,new_domains
33360,c73343e8-395d-40d0-854a-529d11c4e194,천문학자들은 어떤 대상을 연구하나요?,"천문학자와 생물학자는 과학의 다른 영역을 연구합니다. 천문학자들은 하늘에 있는 매우 멀리 떨어진 천체를 관찰하고, 생물학자들은 극도로 작은 객체를 연구합니다. 이 두 분야의 연구자들은 다른 대상을 다루지만, 그들의 연구 방법에는 공통점이 있습니다. 천문학자와 생물학자들은 모두 광학 장치를 사용하여 발견을 합니다. 광학 장치는 망원경, 현미경 등 다양한 형태로 사용되며, 이를 통해 더 나은 관찰과 연구가 가능해집니다. 따라서 천문학자와 생물학자들은 공통적으로 광학 장치를 활용하여 자신들의 분야에서 발견을 이루고 있습니다.",ko_ai2_arc__ARC_Challenge__train,"astronomy, college_biology"
33361,c73343e8-395d-40d0-854a-529d11c4e194,생물학자들은 어떤 객체를 연구하나요?,"천문학자와 생물학자는 과학의 다른 영역을 연구합니다. 천문학자들은 하늘에 있는 매우 멀리 떨어진 천체를 관찰하고, 생물학자들은 극도로 작은 객체를 연구합니다. 이 두 분야의 연구자들은 다른 대상을 다루지만, 그들의 연구 방법에는 공통점이 있습니다. 천문학자와 생물학자들은 모두 광학 장치를 사용하여 발견을 합니다. 광학 장치는 망원경, 현미경 등 다양한 형태로 사용되며, 이를 통해 더 나은 관찰과 연구가 가능해집니다. 따라서 천문학자와 생물학자들은 공통적으로 광학 장치를 활용하여 자신들의 분야에서 발견을 이루고 있습니다.",ko_ai2_arc__ARC_Challenge__train,"astronomy, college_biology"
33362,c73343e8-395d-40d0-854a-529d11c4e194,천문학자와 생물학자의 연구 방법에는 어떤 공통점이 있나요?,"천문학자와 생물학자는 과학의 다른 영역을 연구합니다. 천문학자들은 하늘에 있는 매우 멀리 떨어진 천체를 관찰하고, 생물학자들은 극도로 작은 객체를 연구합니다. 이 두 분야의 연구자들은 다른 대상을 다루지만, 그들의 연구 방법에는 공통점이 있습니다. 천문학자와 생물학자들은 모두 광학 장치를 사용하여 발견을 합니다. 광학 장치는 망원경, 현미경 등 다양한 형태로 사용되며, 이를 통해 더 나은 관찰과 연구가 가능해집니다. 따라서 천문학자와 생물학자들은 공통적으로 광학 장치를 활용하여 자신들의 분야에서 발견을 이루고 있습니다.",ko_ai2_arc__ARC_Challenge__train,"astronomy, college_biology"
33363,c73343e8-395d-40d0-854a-529d11c4e194,광학 장치의 예시로 어떤 것들이 있나요?,"천문학자와 생물학자는 과학의 다른 영역을 연구합니다. 천문학자들은 하늘에 있는 매우 멀리 떨어진 천체를 관찰하고, 생물학자들은 극도로 작은 객체를 연구합니다. 이 두 분야의 연구자들은 다른 대상을 다루지만, 그들의 연구 방법에는 공통점이 있습니다. 천문학자와 생물학자들은 모두 광학 장치를 사용하여 발견을 합니다. 광학 장치는 망원경, 현미경 등 다양한 형태로 사용되며, 이를 통해 더 나은 관찰과 연구가 가능해집니다. 따라서 천문학자와 생물학자들은 공통적으로 광학 장치를 활용하여 자신들의 분야에서 발견을 이루고 있습니다.",ko_ai2_arc__ARC_Challenge__train,"astronomy, college_biology"
33364,c73343e8-395d-40d0-854a-529d11c4e194,천문학자들이 사용하는 장치는 무엇인가요?,"천문학자와 생물학자는 과학의 다른 영역을 연구합니다. 천문학자들은 하늘에 있는 매우 멀리 떨어진 천체를 관찰하고, 생물학자들은 극도로 작은 객체를 연구합니다. 이 두 분야의 연구자들은 다른 대상을 다루지만, 그들의 연구 방법에는 공통점이 있습니다. 천문학자와 생물학자들은 모두 광학 장치를 사용하여 발견을 합니다. 광학 장치는 망원경, 현미경 등 다양한 형태로 사용되며, 이를 통해 더 나은 관찰과 연구가 가능해집니다. 따라서 천문학자와 생물학자들은 공통적으로 광학 장치를 활용하여 자신들의 분야에서 발견을 이루고 있습니다.",ko_ai2_arc__ARC_Challenge__train,"astronomy, college_biology"
33365,c73343e8-395d-40d0-854a-529d11c4e194,생물학자들이 사용하는 장치는 무엇인가요?,"천문학자와 생물학자는 과학의 다른 영역을 연구합니다. 천문학자들은 하늘에 있는 매우 멀리 떨어진 천체를 관찰하고, 생물학자들은 극도로 작은 객체를 연구합니다. 이 두 분야의 연구자들은 다른 대상을 다루지만, 그들의 연구 방법에는 공통점이 있습니다. 천문학자와 생물학자들은 모두 광학 장치를 사용하여 발견을 합니다. 광학 장치는 망원경, 현미경 등 다양한 형태로 사용되며, 이를 통해 더 나은 관찰과 연구가 가능해집니다. 따라서 천문학자와 생물학자들은 공통적으로 광학 장치를 활용하여 자신들의 분야에서 발견을 이루고 있습니다.",ko_ai2_arc__ARC_Challenge__train,"astronomy, college_biology"
33366,c73343e8-395d-40d0-854a-529d11c4e194,광학 장치를 사용하면 어떤 이점이 있나요?,"천문학자와 생물학자는 과학의 다른 영역을 연구합니다. 천문학자들은 하늘에 있는 매우 멀리 떨어진 천체를 관찰하고, 생물학자들은 극도로 작은 객체를 연구합니다. 이 두 분야의 연구자들은 다른 대상을 다루지만, 그들의 연구 방법에는 공통점이 있습니다. 천문학자와 생물학자들은 모두 광학 장치를 사용하여 발견을 합니다. 광학 장치는 망원경, 현미경 등 다양한 형태로 사용되며, 이를 통해 더 나은 관찰과 연구가 가능해집니다. 따라서 천문학자와 생물학자들은 공통적으로 광학 장치를 활용하여 자신들의 분야에서 발견을 이루고 있습니다.",ko_ai2_arc__ARC_Challenge__train,"astronomy, college_biology"
33367,c73343e8-395d-40d0-854a-529d11c4e194,천문학자와 생물학자는 어떤 분야의 연구자들인가요?,"천문학자와 생물학자는 과학의 다른 영역을 연구합니다. 천문학자들은 하늘에 있는 매우 멀리 떨어진 천체를 관찰하고, 생물학자들은 극도로 작은 객체를 연구합니다. 이 두 분야의 연구자들은 다른 대상을 다루지만, 그들의 연구 방법에는 공통점이 있습니다. 천문학자와 생물학자들은 모두 광학 장치를 사용하여 발견을 합니다. 광학 장치는 망원경, 현미경 등 다양한 형태로 사용되며, 이를 통해 더 나은 관찰과 연구가 가능해집니다. 따라서 천문학자와 생물학자들은 공통적으로 광학 장치를 활용하여 자신들의 분야에서 발견을 이루고 있습니다.",ko_ai2_arc__ARC_Challenge__train,"astronomy, college_biology"
33368,c73343e8-395d-40d0-854a-529d11c4e194,천문학자와 생물학자가 연구하는 대상은 어떻게 다르나요?,"천문학자와 생물학자는 과학의 다른 영역을 연구합니다. 천문학자들은 하늘에 있는 매우 멀리 떨어진 천체를 관찰하고, 생물학자들은 극도로 작은 객체를 연구합니다. 이 두 분야의 연구자들은 다른 대상을 다루지만, 그들의 연구 방법에는 공통점이 있습니다. 천문학자와 생물학자들은 모두 광학 장치를 사용하여 발견을 합니다. 광학 장치는 망원경, 현미경 등 다양한 형태로 사용되며, 이를 통해 더 나은 관찰과 연구가 가능해집니다. 따라서 천문학자와 생물학자들은 공통적으로 광학 장치를 활용하여 자신들의 분야에서 발견을 이루고 있습니다.",ko_ai2_arc__ARC_Challenge__train,"astronomy, college_biology"
33369,c73343e8-395d-40d0-854a-529d11c4e194,광학 장치가 발견에 미치는 영향은 무엇인가요?,"천문학자와 생물학자는 과학의 다른 영역을 연구합니다. 천문학자들은 하늘에 있는 매우 멀리 떨어진 

In [10]:
data2_expanded = data2.copy()
data2_expanded['new_domains'] = data2_expanded['new_domains'].str.split(', ')
data2_expanded = data2_expanded.explode('new_domains').reset_index(drop=True)


In [11]:
data2_expanded[data2_expanded['docid']=='c73343e8-395d-40d0-854a-529d11c4e194']

,docid,question,content,src,new_domains
32520,c73343e8-395d-40d0-854a-529d11c4e194,천문학자들은 어떤 대상을 연구하나요?,천문학자와 생물학자는 과학의 다른 영역을 연구합니다. 천문학자들은 하늘에 있는 매우...,ko_ai2_arc__ARC_Challenge__train,astronomy
32521,c73343e8-395d-40d0-854a-529d11c4e194,천문학자들은 어떤 대상을 연구하나요?,천문학자와 생물학자는 과학의 다른 영역을 연구합니다. 천문학자들은 하늘에 있는 매우...,ko_ai2_arc__ARC_Challenge__train,college_biology
32522,c73343e8-395d-40d0-854a-529d11c4e194,생물학자들은 어떤 객체를 연구하나요?,천문학자와 생물학자는 과학의 다른 영역을 연구합니다. 천문학자들은 하늘에 있는 매우...,ko_ai2_arc__ARC_Challenge__train,astronomy
32523,c73343e8-395d-40d0-854a-529d11c4e194,생물학자들은 어떤 객체를 연구하나요?,천문학자와 생물학자는 과학의 다른 영역을 연구합니다. 천문학자들은 하늘에 있는 매우...,ko_ai2_arc__ARC_Challenge__train,college_biology
32524,c73343e8-395d-40d0-854a-529d11c4e194,천문학자와 생물학자의 연구 방법에는 어떤 공통점이 있나요?,천문학자와 생물학자는 과학의 다른 영역을 연구합니다. 천문학자들은 하늘에 있는 매우...,ko_ai2_arc__ARC_Challenge__train,astronomy
32525,c73343e8-395d-40d0-854a-529d11c4e194,천문학자와 생물학자의 연구 방법에는 어떤 공통점이 있나요?,천문학자와 생물학자는 과학의 다른 영역을 연구합니다. 천문학자들은 하늘에 있는 매우...,ko_ai2_arc__ARC_Challenge__train,college_biology
32526,c73343e8-395d-40d0-854a-529d11c4e194,광학 장치의 예시로 어떤 것들이 있나요?,천문학자와 생물학자는 과학의 다른 영역을 연구합니다. 천문학자들은 하늘에 있는 매우...,ko_ai2_arc__ARC_Challenge__train,astronomy
32527,c73343e8-395d-40d0-854a-529d11c4e194,광학 장치의 예시로 어떤 것들이 있나요?,천문학자와 생물학자는 과학의 다른 영역을 연구합니다. 천문학자들은 하늘에 있는 매우...,ko_ai2_arc__ARC_Challenge__train,college_biology
32528,c73343e8-395d-40d0-854a-529d11c4e194,천문학자들이 사용하는 장치는 무엇인가요?,천문학자와 생물학자는 과학의 다른 영역을 연구합니다. 천문학자들은 하늘에 있는 매우...,ko_ai2_arc__ARC_Challenge__train,astronomy
32529,c73343e8-395d-40d0-854a-529d11c4e194,천문학자들이 사용하는 장치는 무엇인가요?,천문학자와 생물학자는 과학의 다른 영역을 연구합니다. 천문학자들은 하늘에 있는 매우...,ko_ai2_arc__ARC_Challenge__train,college_biology


In [12]:
data2_expanded.shape

(41650, 5)

In [13]:
data2_expanded.new_domains.unique()

array(['human_aging', 'medical_genetics', 'high_school_biology',
       'college_chemistry', 'college_physics', 'conceptual_physics',
       'global_facts', 'computer_security', 'high_school_chemistry',
       'anatomy', 'nutrition', 'human_sexuality', 'astronomy',
       'high_school_computer_science', 'virology',
       'electrical_engineering', 'college_medicine', 'college_biology',
       'college_computer_science', 'high_school_physics', 'geology',
       'art', 'college_science', 'safety', 'civil_engineering',
       'new_technology_in_industry', 'engineering', 'data_visualization',
       'environmental_science', 'music_performance', 'logistics', 'None',
       'construction_tools', 'safety_education', 'investigation_records'],
      dtype=object)

In [14]:
data2 = data2_expanded[~data2_expanded['new_domains'].isin(['None'])]

In [15]:
from itertools import product
from itertools import combinations


def create_labelled_pairs(data):
    # 데이터를 도메인별로 그룹화
    grouped_by_domain = data.groupby('new_domains')
    positive_pairs = []
    negative_pairs = []

    # 도메인별로 긍정 쌍 생성
    for domain, group in grouped_by_domain:
        grouped_by_doc = group.groupby('docid')
        # 동일 문서 내의 질문들로 긍정적인 쌍 생성
        for docid, questions_in_doc in grouped_by_doc:
            questions = list(questions_in_doc['question'])
            for q1, q2 in combinations(questions, 2):
                positive_pairs.append(InputExample(texts=[q1, q2], label=1))
        
        # 도메인 내 다른 문서 간 부정적인 쌍 생성
        docids = list(grouped_by_doc.groups.keys())
        for i in range(len(docids)):
            for j in range(i + 1, len(docids)):
                questions1 = list(grouped_by_doc.get_group(docids[i])['question'])
                questions2 = list(grouped_by_doc.get_group(docids[j])['question'])
                for q1, q2 in product(questions1, questions2):
                    negative_pairs.append(InputExample(texts=[q1, q2], label=0))

    # # 다른 도메인 간 부정적인 쌍 생성
    # domains = list(grouped_by_domain.groups.keys())
    # for i in range(len(domains)):
    #     for j in range(i + 1, len(domains)):
    #         questions1 = list(grouped_by_domain.get_group(domains[i])['question'])
    #         questions2 = list(grouped_by_domain.get_group(domains[j])['question'])
    #         for q1, q2 in product(questions1, questions2):
    #             negative_pairs.append(InputExample(texts=[q1, q2], label=0))

    return positive_pairs + negative_pairs
    
# 데이터 생성
training_pairs = create_labelled_pairs(data2)


In [16]:
len(training_pairs)

60706280

In [17]:
from collections import defaultdict
import random

# 도메인별로 데이터 그룹화
domain_groups = defaultdict(list)
for pair in training_pairs:
    domain = pair.domain  # domain이 있다고 가정
    domain_groups[domain].append(pair)

# 각 도메인에서 일정 비율로 샘플링
sample_ratio = 0.01  # 1% 샘플링
sampled_training_pairs = []
for domain, pairs in domain_groups.items():
    sampled_pairs = random.sample(pairs, int(len(pairs) * sample_ratio))
    sampled_training_pairs.extend(sampled_pairs)

print(f"Sampled training pairs length: {len(sampled_training_pairs)}")


AttributeError: 'InputExample' object has no attribute 'domain'

: 

In [149]:
# Print the first 10 pairs to see their content and labels
for example in training_pairs[:30]:
    print("Query:", example.texts[0])
    print("Document:", example.texts[1])
    print("Label:", example.label)
    print("-" * 30)


Query: 아니타는 어떤 활동을 했나요?
Document: 아니타가 달리기를 마친 후 어떤 변화가 있었나요?
Label: 1
------------------------------
Query: 아니타는 어떤 활동을 했나요?
Document: 심장 박동수는 어떻게 변화하나요?
Label: 1
------------------------------
Query: 아니타는 어떤 활동을 했나요?
Document: 아니타의 호흡 속도를 조절하는 기관계는 무엇인가요?
Label: 1
------------------------------
Query: 아니타는 어떤 활동을 했나요?
Document: 신경과 내분비 시스템의 역할은 무엇인가요?
Label: 1
------------------------------
Query: 아니타는 어떤 활동을 했나요?
Document: 아니타의 신체 기능은 어떤 시스템에 의해 조절되나요?
Label: 1
------------------------------
Query: 아니타는 어떤 활동을 했나요?
Document: 달리기 후 아니타가 더 많은 무엇을 들이마셨나요?
Label: 1
------------------------------
Query: 아니타는 어떤 활동을 했나요?
Document: 호흡 속도와 심장 박동수는 어떻게 조절되나요?
Label: 1
------------------------------
Query: 아니타는 어떤 활동을 했나요?
Document: 아니타의 활동 수행에 필요한 조절은 어떤 시스템에 의해 이루어지나요?
Label: 1
------------------------------
Query: 아니타는 어떤 활동을 했나요?
Document: 기관계는 어떤 시스템에 의해 조절되나요?
Label: 1
------------------------------
Query: 아니타가 달리기를 마친 후 어떤 변화가 있었나요?
Document: 심장 박동수는 어떻게 변화하나요?
Label: 1
---------------

In [151]:
for example in training_pairs[30:50]:
    print("Query:", example.texts[0])
    print("Document:", example.texts[1])
    print("Label:", example.label)
    print("-" * 30)

Query: 신경과 내분비 시스템의 역할은 무엇인가요?
Document: 아니타의 신체 기능은 어떤 시스템에 의해 조절되나요?
Label: 1
------------------------------
Query: 신경과 내분비 시스템의 역할은 무엇인가요?
Document: 달리기 후 아니타가 더 많은 무엇을 들이마셨나요?
Label: 1
------------------------------
Query: 신경과 내분비 시스템의 역할은 무엇인가요?
Document: 호흡 속도와 심장 박동수는 어떻게 조절되나요?
Label: 1
------------------------------
Query: 신경과 내분비 시스템의 역할은 무엇인가요?
Document: 아니타의 활동 수행에 필요한 조절은 어떤 시스템에 의해 이루어지나요?
Label: 1
------------------------------
Query: 신경과 내분비 시스템의 역할은 무엇인가요?
Document: 기관계는 어떤 시스템에 의해 조절되나요?
Label: 1
------------------------------
Query: 아니타의 신체 기능은 어떤 시스템에 의해 조절되나요?
Document: 달리기 후 아니타가 더 많은 무엇을 들이마셨나요?
Label: 1
------------------------------
Query: 아니타의 신체 기능은 어떤 시스템에 의해 조절되나요?
Document: 호흡 속도와 심장 박동수는 어떻게 조절되나요?
Label: 1
------------------------------
Query: 아니타의 신체 기능은 어떤 시스템에 의해 조절되나요?
Document: 아니타의 활동 수행에 필요한 조절은 어떤 시스템에 의해 이루어지나요?
Label: 1
------------------------------
Query: 아니타의 신체 기능은 어떤 시스템에 의해 조절되나요?
Document: 기관계는 어떤 시스템에 의해 조절되나요?
Label: 1
----------------

In [ ]:
# https://github.com/UKPLab/sentence-transformers/blob/master/examples/training/cross-encoder/training_stsbenchmark.py

In [ ]:
import logging
import math
from datetime import datetime
import torch
from torch.utils.data import DataLoader
from torch.nn import BCEWithLogitsLoss
from sentence_transformers import LoggingHandler
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CECorrelationEvaluator
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
from tqdm import tqdm  # Import tqdm for the progress bar

# Setup logging
logging.basicConfig(format="%(asctime)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S", level=logging.INFO, handlers=[LoggingHandler()])

# Define model parameters
model_name = "jhgan/ko-sroberta-multitask"
train_batch_size = 32
num_epochs = 4
model_save_path = f"output/training_{model_name.replace('/', '-')}_binary_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"

# Initialize the model and tokenizer
model = CrossEncoder(model_name, num_labels=1)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Split the data into train, validation, and test sets
train_val_pairs, test_pairs = train_test_split(training_pairs, test_size=0.2, random_state=42)
train_pairs, validation_pairs = train_test_split(train_val_pairs, test_size=0.2, random_state=42)

# Custom collate function to tokenize the data
def collate_fn(batch):
    texts = [pair.texts for pair in batch]
    labels = [pair.label for pair in batch]
    
    # Tokenize the texts
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt", max_length=512)
    
    # Convert labels to tensors
    labels = torch.tensor(labels, dtype=torch.float)
    
    return inputs, labels

# Create DataLoader for train, validation, and test sets with custom collate function
train_dataloader = DataLoader(train_pairs, shuffle=True, batch_size=train_batch_size, collate_fn=collate_fn)
validation_dataloader = DataLoader(validation_pairs, shuffle=False, batch_size=train_batch_size, collate_fn=collate_fn)
test_dataloader = DataLoader(test_pairs, shuffle=False, batch_size=train_batch_size, collate_fn=collate_fn)

# Prepare evaluator with validation set
dev_samples = [pair for pair in validation_pairs]
evaluator = CECorrelationEvaluator.from_input_examples(dev_samples, name="aug-dev")

# Configure warmup steps (10% of training steps)
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1)

# Loss function: BCEWithLogitsLoss (which includes sigmoid activation)
loss_fn = BCEWithLogitsLoss()

# Train the model using the fit method with tqdm progress bar
logging.info("Starting training...")
import torch.optim as optim

# Define optimizer
optimizer = optim.AdamW(model.model.parameters(), lr=2e-5)

# Training loop
device = model.model.device  # Access the underlying model's device

for epoch in range(num_epochs):
    model.model.train()  # Ensure the model is in training mode
    progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}", leave=False)

    for step, (inputs, labels) in enumerate(progress_bar):
        # Move inputs and labels to the same device as the model
        inputs = {key: value.to(device) for key, value in inputs.items()}
        labels = labels.to(device)

        # Manually compute outputs and update progress bar
        outputs = model.model(**inputs).logits
        loss = torch.nn.BCEWithLogitsLoss()(outputs.view(-1), labels)

        # Zero gradients, backward pass, and optimizer step
        optimizer.zero_grad()  # Manually zero the gradients
        loss.backward()  # Backpropagation
        optimizer.step()  # Optimizer step to update weights

        # Update the progress bar with the current loss
        progress_bar.set_postfix({'loss': loss.item()})

    # Save the model at the end of each epoch
    epoch_save_path = f"{model_save_path}-epoch-{epoch+1}"
    model.save(epoch_save_path)
    logging.info(f"Model saved after epoch {epoch+1} to {epoch_save_path}")


# Save the final model after all epochs
model.save(model_save_path + "-final")
logging.info(f"Training complete. Final model saved to {model_save_path + '-final'}")


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at jhgan/ko-sroberta-multitask and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1:   1%|          | 12792/1214126 [7:54:47<705:19:24,  2.11s/it, loss=0.00287] 

In [ ]:
# aug_data_3.5turbo1106.jsonl: 1.0으로 PAIR: MAP 0.4955 MRR 0.4955
# aug_data_3.5turbo1106.jsonl: 1.0,-1.0 위 코드로 PAIR: MAP 0.7197 MRR 0.7242
# 